In [1]:
import os
import pandas as pd
import spacy
import re
from fuzzywuzzy import fuzz
# Load the English language model in spaCy
nlp = spacy.load('en_core_web_sm')

In [2]:
def get_advances_data():
    
    adv = pd.read_excel('author_advance_dataset.xlsx', index_col=0)
    adv = adv[~adv['Advance'].isna()]
    deals = {
    'NICE' : (1+49000)/2,
    'VERY NICE' : (50000 + 99000)/2,
    'GOOD' : (100000 + 250000)/2,
    'SIGNIFICANT' : (251000 + 499000)/2,
    'MAJOR' : (500000 + 750000)/2
    }
    adv['Advance'] = adv['Advance'].replace(deals)

    adv.reset_index(inplace=True)
    adv.rename(columns={'index': 'advance_id'}, inplace=True) 
    return adv

def get_printbook_title_data():
    
    book = pd.read_csv('books/booktitle_printbooks_new.csv')
    book = book[['author','title','isbn13', 'datepublished']]
    book['datepublished']  = pd.to_datetime(book['datepublished'], errors = 'coerce')
    #book.rename(columns={'datepublished' : 'datepublished_booktitle_data'})
    
    #Get only the first published date for a given isbn13
    book = book.sort_values('datepublished')
    book = book.groupby('isbn13').first().reset_index()
    
    return book

def get_Ebook_title_data():
    
    book = pd.read_csv('books/booktitle_ebook_new.csv')
    book = book[['author','title','isbn13', 'datepublished']]
    book['datepublished']  = pd.to_datetime(book['datepublished'], errors = 'coerce')
    #book.rename(columns={'datepublished' : 'datepublished_booktitle_data'})
    
    #Get only the first published date for a given isbn13
    book = book.sort_values('datepublished')
    book = book.groupby('isbn13').first().reset_index()
    
    return book

def extract_author_name_1(text):
    doc = nlp(text)
    for entity in doc.ents:
        if entity.label_ == 'PERSON' and ("'s" in entity.text or "'" in entity.text):
            return entity.text
    return None


def extract_author_name_2(text):
    doc = nlp(text)
    for entity in doc.ents:
        if entity.label_ == 'PERSON' :#and ("'s" in entity.text or "'" in entity.text):
            return entity.text
    return None

def author_extraction(adv):
    adv['Author_extracted'] = adv['All'].apply(lambda x: extract_author_name_1(x))
    adv.loc[(adv['Author_extracted'].isna()) & ((adv['Author(s)'] == 'NONE') | (adv['Author(s)'] == 'All')), 'Author_extracted'] = adv.loc[(adv['Author_extracted'].isna()) & ((adv['Author(s)'] == 'NONE') | (adv['Author(s)'] == 'All')), 'All'].apply(lambda x: extract_author_name_2(x))
    adv.loc[(adv['Author_extracted'].isna()) & (adv['Author(s)'] != 'NONE'), 'Author_extracted'] = adv.loc[(adv['Author_extracted'].isna()) & (adv['Author(s)'] != 'NONE'), 'Author(s)'] 
    adv['Author_extracted'] = adv['Author_extracted'].str.replace("'s", "")
    adv['Author_extracted'] = adv['Author_extracted'].str.replace("'", "")
    adv['Author_extracted'] = adv['Author_extracted'].str.lower()
    return adv


def levenshtein_ratio(s1, s2):
    return fuzz.token_sort_ratio(s1, s2)

def get_GAPI_books():
    d1 = pd.read_csv('AuthorCrosswalkScraper/Google_Author_crosswalk_adv.csv')
    d1['Author'] = d1['Author'].astype(str)
    d2 = pd.read_csv('AuthorCrosswalkScraper/Google_Author_crosswalk_adv_17440.csv')
    d2['Author'] = d2['Author'].astype(str)
    d3 = pd.read_csv('AuthorCrosswalkScraper/Unmatched_extraction_of_authors.csv')
    d3['Author'] = d3['Author'].astype(str)
    d4 = pd.read_csv('AuthorCrosswalkScraper/Google_Author_crosswalk_adv_x.csv')
    d4['Author'] = d4['Author'].astype(str)
    d = pd.concat([d1,d2],axis=0)
    d = pd.concat([d,d3], axis=0)
    d = pd.concat([d,d4], axis=0)
    d['Author'] = d['Author'].astype(str)
    d['Author'] = d['Author'].str.replace("[","")
    d['Author'] = d['Author'].str.replace("]","")
    d['Author'] = d['Author'].str.replace("'","")
    d['author_l'] = d['Author'].str.lower()
    d = d.drop(columns='Unnamed: 0')
    d.columns = ['author','title','datepublished','isbn13','author_l']
    
    return d

def merge_book_advances(book, adv):
    
    book_adv = pd.merge(book, adv, left_on='author_l', right_on='Author_extracted')
    book_adv['fuzz_ratio'] = book_adv.apply(lambda x : levenshtein_ratio(str(x['Title']).lower(), str(x['title']).lower()), axis=1)
    
    extract_all_caps = lambda text: ' '.join(re.findall(r'\b[A-Z]{2,}\b', text))
    book_adv['title_extracted'] = book_adv['All'].apply(extract_all_caps)
    
    book_adv = book_adv[~((book_adv['Title'] == 'NONE') & (book_adv['title_extracted']=="")) ]
    book_adv.loc[book_adv['Title'] == 'NONE', 'fuzz_ratio'] = book_adv.loc[book_adv['Title'] == 'NONE'].apply(lambda x : levenshtein_ratio(str(x['title_extracted']).lower(), str(x['title']).lower()), axis=1)
    
    book_adv['datepublished'] = book_adv['datepublished'].astype(str)
    book_adv['datepublished'] = pd.to_datetime(book_adv['datepublished'], errors='coerce')
    book_adv['datepublished'] = book_adv['datepublished'].apply(lambda x: x.date() if pd.notnull(x) and hasattr(x, 'date') else pd.NaT if pd.isnull(x) else pd.to_datetime(x[:10]).date() if len(x) == 10 else pd.to_datetime(x + '-12-31').date() if len(x) == 4 else pd.to_datetime(x + '-01').date() + pd.offsets.MonthEnd() if len(x) == 7 else x)
    '''
    book_adv_grouped = book_adv.groupby('advance_id', as_index=False).apply(lambda x : x.loc[x['fuzz_ratio'].idxmax()])
    book_adv_grouped = book_adv_grouped.groupby('isbn13', as_index=False).apply(lambda x : x.loc[x['fuzz_ratio'].idxmax()])
    '''
    book_adv_grouped = book_adv
    book_adv_grouped['datepublished'] = pd.to_datetime(book_adv_grouped['datepublished'])
    return book_adv_grouped

In [21]:
def any_merge(advance_data_this_row,sales_data):
    sales_advances = pd.DataFrame(columns=list(advance_data_this_row.columns)+ list(sales_data))
    isbn13_advance_data = advance_data_this_row['isbn13']
    
    for isbn in isbn13_advance_data:
        if isbn in df['isbn13']:
            sales_advances = pd.concat[[sales_advances, isbn13_advance_data[isbn13_advance_data['isbn13']==isbn]]]
    
    return sales_advances
    
    

def df_sales_merge():
    sales = pd.read_csv('sales/ebook_sales_aggregate.csv', index_col=0)
    sales = sales.groupby('isbn13',as_index=False).agg({'totalrevenue':'sum'}).reset_index()
    sale_title = pd.read_csv('AuthorCrosswalkScraper/All_isbn13_for_sales_by_Author_Agg.csv',index_col=0)
    sale_title = sale_title.rename(columns={
    'Author': 'author',
    'Title' : 'title',
    'PublishedDate': 'datepublished',
    'ISBN13' : 'isbn13'
    })
    sale_title['author'] = sale_title['author'].str.replace("'", "").str.replace("[", "").str.replace("]", "")
    sale_title = sale_title.drop_duplicates()
    #print(sales.columns, sale_title.columns)
    
    salesx = pd.merge(sale_title, sales, on='isbn13', how='outer')
    #salesx = salesx.drop(columns=['datepublished_x'])
    salesx = salesx.rename(columns={'datepublished_y' : 'datepublished',
                                   'title': 'sales_title',
                                   'author': 'sales_author'})
    mf = pd.merge(salesx, df , on = 'isbn13', how='outer')
    mf['sales_title'] = mf['sales_title'].fillna(mf['title'])
    mf['sales_author'] = mf['sales_author'].fillna(mf['author'])
    print(mf['advance_id'].value_counts())
    print(mf.columns)
    mf = mf.groupby(['sales_author','sales_title'], as_index=False).agg({'totalrevenue':'sum',
                                                                                  'Advance':'max'})
    mf = mf.loc[ (~mf['Advance'].isna()) & (mf['totalrevenue']>0)]
    return mf
    '''
    
    salesx = pd.merge(sale_title, salesx, on='author')
    salesx = salesx.drop_duplicates()
    salesx = salesx.drop(columns=['datepublished_x'])
    salesx = salesx.rename(columns={'datepublished_y' : 'datepublished'})
    #salex = pd.merge(salex)
    '''
    mf = pd.merge(salesx, df , left_on='isbn13', right_on = 'isbn13', how='outer')
    print(mf['advance_id'].value_counts())
    print(mf.columns)
    mf = mf.groupby(['sales_author','sales_title'], as_index=False).agg({'totalrevenue':'sum'})
    mf = mf.loc[mf['totalrevenue'] >0]
    
    #mf = pd.merge(mf, df, left_on=['sales_author','sales_title'], right_on=['author', 'title'])
    return mf
    
    #mf2 = pd.merge(salesx, df, left_on='isbn13_y', right_on='isbn13', how='right')
    
    #mf = pd.merge(salesx, df, left_on='isbn13_x', right_on='isbn13')
    #mf = pd.concat([mf1,mf2])
    
    mf = mf.drop(columns=['title','isbn13' ])
    mf = mf.drop_duplicates()
    #print(mf.shape, mf.columns)
    print(mf.columns, mf.head())
        
    kf = pd.merge(mf,sale_title, left_on=['author_x','title_x'], right_on=['author','title'], how='inner')
    print(kf.columns,kf.shape, kf.head())
    kf = kf.groupby(['author_x','title_x','datepublished_x'], as_index=False).agg({'totalrevenue':'sum'})
    #kf = kf.groupby('datepublished_x', as_index=False).agg({'totalrevenue':'sum'})
    
    '''
    salesx = pd.merge(sales, sale_title, on='isbn13')
    salesx = salesx.drop(columns=['datepublished_y'])
    salesx = salesx.rename(columns={'datepublished_x' : 'datepublished'})
    print(salesx.columns)
    
    salesxx = pd.merge( salesx,sale_title, on=['author','title'])
    salesxx = salesxx.drop(columns=['datepublished_y'])
    salesxx = salesxx.rename(columns={'datepublished_x' : 'datepublished'})
    print(salesxx.columns)
    
    mf1 = pd.merge(salesxx, df, left_on='isbn13_x', right_on='isbn13')
    mf2 = pd.merge(salesxx, df, left_on='isbn13_y', right_on='isbn13')
    mf = pd.concat([mf1,mf2])
    
    '''
    '''
    sales_x = pd.merge(df, sale_title, on=['isbn13'])
    #sales_x = pd.merge(sales_x, sale_title, on=['author','title'])
    #sales_x = sales_x.drop(columns=['datepublished_y','datepublished'])
    print(sales_x.columns, sales_x.shape)
    mm = pd.merge(sales_x, sale_title, left_on=['author_x','title_x'], right_on=['author','title'], how='inner')
    #mn = pd.merge(mm,sales, on='isbn13')
    mm =mm.drop(columns=['datepublished_y','datepublished'])
    print(mm.columns)
    m1 = pd.merge(mm, sales, left_on=['isbn13_x'] , right_on=['isbn13'] , how='inner')
    m2 = pd.merge(mm, sales, left_on=['isbn13_y'] , right_on=['isbn13'] , how='inner')
    mn = pd.concat([m1,m2])
    mn = mn.drop_duplicates()
    #mm = pd.merge(sales_x, df, on= ['author','title'])
    '''
    return salesx
    
def df_sales_merge_time_series():
    i=0
    sale_title = pd.read_csv('AuthorCrosswalkScraper/All_isbn13_for_sales_by_Author_Agg.csv',index_col=0)
    sale_title = sale_title.rename(columns={
    'Author': 'author',
    'Title' : 'title',
    'PublishedDate': 'datepublished',
    'ISBN13' : 'isbn13'
    })
    sale_title['author'] = sale_title['author'].str.replace("'", "").str.replace("[", "").str.replace("]", "")
    sale_title = sale_title.drop_duplicates()
    for file in os.listdir('sales/printbook/'):
        sales = pd.read_csv(f'sales/printbook/{file}')
        sales = sales.groupby('isbn13',as_index=False).agg({'totalrevenue':'sum'}).reset_index()
        
        
        salesx = pd.merge(sale_title, sales, on='isbn13', how='outer')
        #salesx = salesx.drop(columns=['datepublished_x'])
        salesx = salesx.rename(columns={'datepublished_y' : 'datepublished',
                                       'title': 'sales_title',
                                       'author': 'sales_author'})
        mf = pd.merge(salesx, df , left_on='isbn13', right_on = 'isbn13', how='outer')
        #print(mf['advance_id'].value_counts())
        #print(mf.columns)
        mf = mf.groupby(['sales_author','sales_title','isbn13'], as_index=False).agg({'totalrevenue':'sum'})
        mf = mf.loc[mf['totalrevenue'] >0]
        mf['sales_month_year'] = file[-12:-6]
        mf['sales_month_year'] = pd.to_datetime(mf['sales_month_year'], format='%Y%m')
        if i==0:
            xf = mf
            i+=1
            continue
        xf = pd.concat([xf, mf])
        print(f"{file} - processed")
    #mf = pd.merge(mf, df, left_on=['sales_author','sales_title'], right_on=['author', 'title'])
    return xf

In [22]:
df_sales_merge()

advance_id
160215.0    1
128031.0    1
136221.0    1
129130.0    1
160789.0    1
           ..
155392.0    1
153317.0    1
154164.0    1
167932.0    1
7031.0      1
Name: count, Length: 15799, dtype: int64
Index(['sales_title', 'sales_author', 'datepublished_x', 'isbn13', 'index',
       'totalrevenue', 'author', 'title', 'datepublished_y', 'author_l',
       'advance_id', 'Rights Category', 'Genre', 'Date', 'Author(s)', 'Title',
       'Publishers', 'Big Publishing House Affilation', 'Advance',
       'Competition', 'Awards', 'Bestseller', 'Self Publishing', 'Debut',
       'Series', 'All', 'Author_extracted', 'fuzz_ratio', 'title_extracted'],
      dtype='object')


,sales_author,sales_title,totalrevenue,Advance
4198,Aaron Blabey,Pig the Star,122877.0,74500.0
4265,Aaron Bobrow-Strain,The Death and Life of Aida Hernandez,724289.0,175000.0
4439,Abbi Glines,After the Game,16239291.0,625000.0
4580,Abbie Greaves,The Silent Treatment,27638.0,375000.0
4737,Abigail Hing Wen,"Loveboat, Taipei",409731.0,625000.0
...,...,...,...,...
358415,Wu Ming-Yi,The Stolen Bicycle,392968.0,24500.5
358755,Yanis Varoufakis,Talking to My Daughter About the Economy,2979612.0,175000.0
359135,Yigal Zur,Death in Shangri-La,59233.0,24500.5
359145,Yiota Giannakopoulou,The Gift of the Greek,409913.0,24500.5


In [4]:
#step 1
d = get_GAPI_books()
#step 2
adv = get_advances_data()
adv = author_extraction(adv)

In [5]:
#step 3
merg = merge_book_advances(d, adv)
#step 4
f = merg[merg['fuzz_ratio']>=95]
f = f.groupby('advance_id', as_index=False).apply(lambda x : x.loc[x['fuzz_ratio'].idxmax()])
f = f.groupby('isbn13', as_index=False).apply(lambda x : x.loc[x['fuzz_ratio'].idxmax()])
print("step 4",f.shape, f['advance_id'].unique().shape, f['isbn13'].unique().shape, f['author_l'].unique().shape)

merg = merg[(~merg['isbn13'].isin(f['isbn13'])) & (~merg['advance_id'].isin(f['advance_id']))]
#step 5
filtered_merg = merg[((merg['datepublished'] - merg['Date']).dt.days <=1460) & ((merg['datepublished'] - merg['Date']).dt.days >=90) & (merg['fuzz_ratio']>=80)]
filtered_merg = filtered_merg.groupby('advance_id', as_index=False).apply(lambda x : x.loc[x['fuzz_ratio'].idxmax()])
filtered_merg = filtered_merg.groupby('isbn13', as_index=False).apply(lambda x : x.loc[x['fuzz_ratio'].idxmax()])
print("step 5",filtered_merg.shape, filtered_merg['advance_id'].unique().shape, filtered_merg['isbn13'].unique().shape, filtered_merg['author_l'].unique().shape)

merg = merg[(~merg['isbn13'].isin(filtered_merg['isbn13'])) & (~merg['advance_id'].isin(filtered_merg['advance_id']))]
#step 6
df = pd.concat([f,filtered_merg], axis=0)
print("step 6 = step 4 + step 5",df.shape, df['advance_id'].unique().shape, df['isbn13'].unique().shape, df['author_l'].unique().shape)
#df = df.drop_duplicates()
#df = df.groupby('advance_id', as_index=False).apply(lambda x : x.loc[x['fuzz_ratio'].idxmax()])
#df = df.groupby('isbn13', as_index=False).apply(lambda x : x.loc[x['fuzz_ratio'].idxmax()])

#step 7
rem = merg[(~merg['isbn13'].isin(df['isbn13'])) & (~merg['advance_id'].isin(df['advance_id']))]
rem = merg[((merg['datepublished'] - merg['Date']).dt.days <=1460) & ((merg['datepublished'] - merg['Date']).dt.days >=90)]
rem = rem.groupby('advance_id', as_index=False).apply(lambda x : x.loc[x['fuzz_ratio'].idxmax()])
rem = rem.groupby('isbn13', as_index=False).apply(lambda x : x.loc[x['fuzz_ratio'].idxmax()])
print("step 7", rem.shape, rem['advance_id'].unique().shape, rem['isbn13'].unique().shape, rem['author_l'].unique().shape)
df = pd.concat([df,rem], axis=0)
print("final merge",df.shape, df['advance_id'].unique().shape, df['isbn13'].unique().shape, df['author_l'].unique().shape)
#df = df.drop_duplicates()
#df = df.groupby('advance_id', as_index=False).apply(lambda x : x.loc[x['fuzz_ratio'].idxmax()])
#df = df.groupby('isbn13', as_index=False).apply(lambda x : x.loc[x['fuzz_ratio'].idxmax()])
#print(df.shape, df['advance_id'].unique().shape, df['isbn13'].unique().shape, df['author_l'].unique().shape)

step 4 (8217, 24) (8217,) (8217,) (7213,)
step 5 (675, 24) (675,) (675,) (650,)
step 6 = step 4 + step 5 (8892, 24) (8892,) (8892,) (7576,)
step 7 (6907, 24) (6907,) (6907,) (5557,)
final merge (15799, 24) (15799,) (15799,) (11810,)


In [43]:
adv

,advance_id,Rights Category,Genre,Date,Author(s),Title,Publishers,Big Publishing House Affilation,Advance,Competition,Awards,Bestseller,Self Publishing,Debut,Series,All,Author_extracted
0,2,International rights,UK Fiction,2023-03-15,Kynpham Sing Nongkynrih,FUNERAL NIGHTS,"['Stefan Tobler', 'And Other Stories', 'Tara T...",NaN,24500.5,NaN,NaN,NaN,NaN,NaN,NaN,Indian writer Kynpham Sing Nongkynrih's FUNERA...,kynpham sing nongkynrih
1,4,International rights,Fiction,2023-03-15,NONE,THE SHADOW MURDERS,"['Fantasy Foundation', 'Ann Huang', 'The Grayh...",NaN,24500.5,NaN,NaN,NaN,NaN,NaN,NaN,"Jussi Adler-Olsen's THE SHADOW MURDERS, to Fan...",jussi adler-olsen
2,7,Fiction,Horror,2023-03-15,Chance Forshee,A TERRIFYING TOME OF TERROR,"['Jennifer Barnes', 'Raw Dog Screaming Press']",NaN,24500.5,NaN,NaN,NaN,NaN,NaN,NaN,"The Curator of Horror, a bookseller, and host ...",chance forshee
3,13,Children's,Picture Book Fiction,2023-03-15,"['Debra Buschman', 'Tom Uleau']",THE KNIGHT & HIS TRUSTY,"['Michele McAvoy', 'The Little Press']",NaN,24500.5,NaN,NaN,NaN,NaN,NaN,NaN,SCBWI Wisconsin Regional Advisor Debra Buschma...,"[debra buschman, tom uleau]"
4,22,Children's,Graphic Novel,2023-03-14,Junepurrr,SUBZERO,"['Desiree Rodriguez', 'Oni Press', 'Britt Sies...",NaN,74500.0,NaN,NaN,NaN,NaN,NaN,NaN,Creator of the webcomic series of the same nam...,junepurrr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34292,168767,Non-fiction,History,2008-01-07,Stefan Aust,BAADER-MEINHOF,"['Cybele Tom', 'Oxford University Press']",NaN,24500.5,NaN,NaN,NaN,NaN,NaN,NaN,Der Spiegel's Editor-in-Chief Stefan Aust's BA...,der spiegel
34293,168773,Children's,Young Adult Fiction,2008-01-07,David Macinnis Gill,SOUL ENCHILADA,"['Virginia Duncan', ""Harper Children's"", 'Rose...",Harper,175000.0,YES,NaN,NaN,NaN,NaN,NaN,"David Macinnis Gill's SOUL ENCHILADA, in which...",david macinnis gill
34294,168780,Non-fiction,Advice/ Relationships/ Self-Help,2008-01-06,NONE,NONE,NONE,NaN,175000.0,YES,NaN,NaN,NaN,NaN,NaN,"Peg Streep's MEAN MOTHERS, exploring the psych...",peg streep
34295,168797,Non-fiction,How-To,2008-01-04,"['Linda Meyers', 'John Meyers']",TOSSED & FOUND,"['Dervla Kelly', 'Stewart, Tabori & Chang', 'S...",NaN,24500.5,NaN,NaN,NaN,NaN,NaN,NaN,Interior designers Linda Meyers and John Meyer...,john meyers


In [34]:
df.columns

Index(['author', 'title', 'datepublished', 'isbn13', 'author_l', 'advance_id',
       'Rights Category', 'Genre', 'Date', 'Author(s)', 'Title', 'Publishers',
       'Big Publishing House Affilation', 'Advance', 'Competition', 'Awards',
       'Bestseller', 'Self Publishing', 'Debut', 'Series', 'All',
       'Author_extracted', 'fuzz_ratio', 'title_extracted'],
      dtype='object')

In [5]:
df.to_csv('data_before_sales_merge.csv')

In [5]:
sales = pd.read_csv('sales/ebook_sales_aggregate.csv', index_col=0)

In [6]:
sale_title = pd.read_csv('AuthorCrosswalkScraper/All_isbn13_for_sales_by_Author_Agg.csv',index_col=0)

In [7]:
sale_title = sale_title.rename(columns={
    'Author': 'author',
    'Title' : 'title',
    'PublishedDate': 'datepublished',
    'ISBN13' : 'isbn13'
})

In [8]:
sale_title['author'] = sale_title['author'].str.replace("'", "").str.replace("[", "").str.replace("]", "")

In [15]:
kx = pd.merge(sales, sale_title, on='isbn13')

In [18]:
kx.columns

Index(['isbn13', 'datepublished_x', 'totalrevenue', 'totalunits', 'price',
       'title', 'author', 'datepublished_y'],
      dtype='object')

In [19]:
kx = pd.merge(kx, sale_title, on=['author', 'title'])

In [30]:
kx['isbn13_y'].unique().shape

(14439,)

In [24]:
kx = kx.rename(columns={'datepublished':'datepublished_z'})

In [31]:
shpa = pd.merge(kx, df, left_on='isbn13_y' , right_on='isbn13')


In [33]:
shpa

,isbn13_x,datepublished_x,totalrevenue,totalunits,price,title_x,author_x,datepublished_y,datepublished_z,isbn13_y,...,Competition,Awards,Bestseller,Self Publishing,Debut,Series,All,Author_extracted,fuzz_ratio,title_extracted
0,9.780062e+12,2015-03-03,7182,18,403.333333,The Memory Key,Liana Liu,2015,2015,9780062306661,...,YES,NaN,NaN,NaN,YES,NaN,University of Minnesota MFA graduate Liana Liu...,liana liu,100,MFA THE MEMORY KEY NA
1,9.780062e+12,2017-12-05,12716194,12413,1224.837576,The Saboteur,Paul Kix,2017,2017,9780062322548,...,NaN,NaN,NaN,NaN,NaN,NaN,"Journalist Paul Kix's THE NOBLE ASSASSIN, the ...",paul kix,53,THE NOBLE ASSASSIN
2,9.780062e+12,2018-05-01,4586380,4281,1068.603214,Not That Bad,Roxane Gay,2018,2018,9780062413505,...,NaN,NaN,NaN,NaN,NaN,NaN,"Roxane Gay's NOT THAT BAD, to Folio (China), i...",roxane gay,100,NOT THAT BAD
3,9.780062e+12,2018-06-03,0,0,1099.000000,Not That Bad,Roxane Gay,2018,2018,9780062413505,...,NaN,NaN,NaN,NaN,NaN,NaN,"Roxane Gay's NOT THAT BAD, to Folio (China), i...",roxane gay,100,NOT THAT BAD
4,9.780062e+12,2023-11-06,0,0,853.000000,Not That Bad,Roxane Gay,2018,2018,9780062413505,...,NaN,NaN,NaN,NaN,NaN,NaN,"Roxane Gay's NOT THAT BAD, to Folio (China), i...",roxane gay,100,NOT THAT BAD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414,9.781985e+12,2020-07-07,69537,81,889.858586,Kill Orbit,Joel Dane,2020,2020,9781984802576,...,YES,NaN,NaN,NaN,NaN,NaN,"Joel Dane's CRY PILOT, BURN CYCLE, and KILL OR...",joel dane,49,CRY PILOT BURN CYCLE KILL ORBIT SF
415,9.781985e+12,2019-05-21,1063507,855,1244.488599,Star-Crossed,Minnie Darke,2019,2019,9781984822840,...,YES,YES,NaN,NaN,NaN,NaN,"Minnie Darke's STAR-CROSSED, pitch as in the v...",minnie darke,100,STAR CROSSED THE ROSIE PROJECT NA UK IPG
416,9.781985e+12,2019-12-31,998451,833,1213.104895,A Woman Makes a Plan,Maye Musk,2019,2019,9781984878519,...,YES,NaN,NaN,NaN,NaN,NaN,"Maye Musk's A WOMAN MAKES A PLAN, to Stefan Ma...",maye musk,100,WOMAN MAKES PLAN
417,9.782214e+12,2018-09-26,35063,20,1880.473684,Un gentleman à Moscou,Amor Towles,2018,2018,9782213704296,...,NaN,NaN,NaN,NaN,NaN,NaN,Rules of Civility author Amor Towles next two ...,amor towles,33,NA UK


In [32]:
shpa.shape

(419, 34)

In [13]:
shpa2 = pd.merge(shpa, sale_title, on='isbn13')

In [14]:
shpa2

,isbn13,datepublished_x,totalrevenue,totalunits,price,author_x,title_x,datepublished_y,author_l,advance_id,...,Self Publishing,Debut,Series,All,Author_extracted,fuzz_ratio,title_extracted,title_y,author_y,datepublished
0,9.780062e+12,2015-03-03,7182,18,403.333333,Liana Liu,The Memory Key,2015-01-01,liana liu,124290,...,NaN,YES,NaN,University of Minnesota MFA graduate Liana Liu...,liana liu,100,MFA THE MEMORY KEY NA,The Memory Key,Liana Liu,2015
1,9.780062e+12,2017-12-05,12716194,12413,1224.837576,Paul Kix,The Saboteur,2017-01-01,paul kix,120953,...,NaN,NaN,NaN,"Journalist Paul Kix's THE NOBLE ASSASSIN, the ...",paul kix,53,THE NOBLE ASSASSIN,The Saboteur,Paul Kix,2017
2,9.780062e+12,2018-05-01,4586380,4281,1068.603214,Roxane Gay,Not That Bad,2018-01-01,roxane gay,39661,...,NaN,NaN,NaN,"Roxane Gay's NOT THAT BAD, to Folio (China), i...",roxane gay,100,NOT THAT BAD,Not That Bad,Roxane Gay,2018
3,9.780062e+12,2018-06-03,0,0,1099.000000,Roxane Gay,Not That Bad,2018-01-01,roxane gay,39661,...,NaN,NaN,NaN,"Roxane Gay's NOT THAT BAD, to Folio (China), i...",roxane gay,100,NOT THAT BAD,Not That Bad,Roxane Gay,2018
4,9.780062e+12,2023-11-06,0,0,853.000000,Roxane Gay,Not That Bad,2018-01-01,roxane gay,39661,...,NaN,NaN,NaN,"Roxane Gay's NOT THAT BAD, to Folio (China), i...",roxane gay,100,NOT THAT BAD,Not That Bad,Roxane Gay,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,9.781985e+12,2020-07-07,69537,81,889.858586,Joel Dane,Kill Orbit,2020-01-01,joel dane,65841,...,NaN,NaN,NaN,"Joel Dane's CRY PILOT, BURN CYCLE, and KILL OR...",joel dane,49,CRY PILOT BURN CYCLE KILL ORBIT SF,Kill Orbit,Joel Dane,2020
275,9.781985e+12,2019-05-21,1063507,855,1244.488599,Minnie Darke,Star-Crossed,2019-01-01,minnie darke,66743,...,NaN,NaN,NaN,"Minnie Darke's STAR-CROSSED, pitch as in the v...",minnie darke,100,STAR CROSSED THE ROSIE PROJECT NA UK IPG,Star-Crossed,Minnie Darke,2019
276,9.781985e+12,2019-12-31,998451,833,1213.104895,Maye Musk,A Woman Makes a Plan,2019-01-01,maye musk,46356,...,NaN,NaN,NaN,"Maye Musk's A WOMAN MAKES A PLAN, to Stefan Ma...",maye musk,100,WOMAN MAKES PLAN,A Woman Makes a Plan,Maye Musk,2019
277,9.782214e+12,2018-09-26,35063,20,1880.473684,Amor Towles,Un gentleman à Moscou,2018-01-01,amor towles,90863,...,NaN,NaN,NaN,Rules of Civility author Amor Towles next two ...,amor towles,33,NA UK,Un gentleman à Moscou,Amor Towles,2018


In [11]:
shpa['isbn13'].unique().shape

(508,)

In [47]:
shpa['author'] = shpa['author'].str.replace("'", "").str.replace("[", "").str.replace("]", "")

In [53]:
ISBN13_j = pd.merge(df, sale_title, on='isbn13')

In [55]:
ISBN13_j.columns

Index(['author_x', 'title_x', 'datepublished_x', 'isbn13', 'author_l',
       'advance_id', 'Rights Category', 'Genre', 'Date', 'Author(s)', 'Title',
       'Publishers', 'Big Publishing House Affilation', 'Advance',
       'Competition', 'Awards', 'Bestseller', 'Self Publishing', 'Debut',
       'Series', 'All', 'Author_extracted', 'fuzz_ratio', 'title_extracted',
       'title_y', 'author_y', 'datepublished_y'],
      dtype='object')

In [57]:
pd.merge(df, sale_title, on=['author','title'])

,author,title,datepublished_x,isbn13_x,author_l,advance_id,Rights Category,Genre,Date,Author(s),...,Bestseller,Self Publishing,Debut,Series,All,Author_extracted,fuzz_ratio,title_extracted,datepublished_y,isbn13_y
0,Madeleine Roux,Asylum,2013-01-01,9780007538256,madeleine roux,134628,Children's,Young Adult Fiction,2012-03-08,Madeleine Roux,...,NaN,NaN,NaN,NaN,SADIE WALKER IS STRANDED author Madeleine Roux...,madeleine roux,100,SADIE WALKER IS STRANDED ASYLUM,2013,9780007538256
1,Jeff Jackson,Mira Corpora,2014-01-01,9780007586370,jeff jackson,126531,Fiction,Debut,2013-01-10,Jeff Jackson,...,NaN,NaN,NaN,NaN,"Jeff Jackson's MIRA CORPORA, the story of an 1...",jeff jackson,100,MIRA CORPORA NA,2014,9780007586370
2,Kendall Ryan,Resisting Her,2015-01-01,9780008134051,kendall ryan,125056,Fiction,Romance,2013-03-02,Kendall Ryan,...,YES,NaN,NaN,NaN,NYT bestselling author Kendall Ryan's RESISTIN...,kendall ryan,100,NYT RESISTING HER FBI NA,2015,9780008134051
3,Dianne Lake,Member of the Family,2018-01-01,9780008274764,dianne lake,81901,Non-fiction,Memoir,2017-01-12,Dianne Lake,...,NaN,NaN,NaN,NaN,Dianne Lake's MEMBER OF THE FAMILY: My Story o...,dianne lake,100,MEMBER OF THE FAMILY,2018,9780008274764
4,Nnedi Okorafor,Who Fears Death,2018-01-01,9780008288747,nnedi okorafor,161284,Fiction,Sci-Fi/ Fantasy,2009-01-28,Nnedi Okorafor,...,NaN,NaN,NaN,NaN,"Nnedi Okorafor's WHO FEARS DEATH?, Nigerian-Am...",nnedi okorafor,100,WHO FEARS DEATH DAW NA,2020,9780756417109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3329,J.R. Johansson,Condenado a muerte,2015-01-01,9789877474404,j.r. johansson,124480,Children's,Young Adult Fiction,2013-03-20,J.R. Johansson,...,NaN,NaN,NaN,NaN,INSOMNIA author J.R. Johansson's untitled new ...,j.r. johansson,23,INSOMNIA,2015,9789877474404
3330,Patrick Rothfuss,A Música do Silêncio,2015-01-01,9789892330723,patrick rothfuss,130549,Fiction,Sci-Fi/ Fantasy,2012-08-08,Patrick Rothfuss,...,YES,NaN,NaN,NaN,New York Times bestselling author of The Name ...,patrick rothfuss,19,DAW,2015,9789892330723
3331,Hailey Edwards,Gray Tidings,2022-01-01,9798360451945,hailey edwards,59350,International rights,Fiction,2018-10-17,NONE,...,NaN,NaN,NaN,NaN,"Hailey Edwards's BAYOU BORN and BONE DRIVEN, t...",hailey edwards,35,BAYOU BORN BONE DRIVEN,2022,9798360451945
3332,Alexandre Dumas,The Black Tulip Annotated,2021-01-01,9798591189884,alexandre dumas,43532,Fiction,General/ Other,2020-01-28,"['Lawrence Ellsworth', 'Alexandre Dumas']",...,NaN,NaN,NaN,NaN,"Translator of THE THREE MUSKETEERS, TWENTY YEA...",alexandre dumas,28,THE THREE MUSKETEERS TWENTY YEARS AFTER THE RE...,2021,9798591189884


In [26]:
fm = pd.merge(shpa, df, left_on='author', right_on = 'author')
fm['datepublished_y'] = pd.to_datetime(fm['datepublished_y'])

In [27]:
fm['fuzz_ratio_1'] = fm.apply(lambda x : levenshtein_ratio(str(x['title_x']).lower(), str(x['title_y']).lower()), axis=1)

In [28]:
fm[fm['fuzz_ratio_1'] >= 70]

,author,title_x,datepublished_x,isbn13_x,datepublished_y,totalrevenue,totalunits,price,title_y,datepublished,...,Awards,Bestseller,Self Publishing,Debut,Series,All,Author_extracted,fuzz_ratio,title_extracted,fuzz_ratio_1
24,Paul Kix,The Saboteur,2017-12-05,9780062322548,2017-12-05,12716194,12413,1224.837576,The Saboteur,2017-01-01,...,NaN,NaN,NaN,NaN,NaN,"Journalist Paul Kix's THE NOBLE ASSASSIN, the ...",paul kix,53,THE NOBLE ASSASSIN,100
49,Erin Entrada Kelly,You Go First,2018-04-10,9780062414205,2018-04-10,1349539,2018,734.842105,You Go First,2018-01-01,...,NaN,NaN,NaN,NaN,NaN,"Author of the forthcoming BLACKBIRD FLY, Erin ...",erin entrada kelly,41,BLACKBIRD FLY ALMOST VIRGIL YOU WERE HERE,100
104,Heather Kaczynski,Dare Mighty Things,2017-10-10,9780062479891,2017-10-10,366170,1069,831.631373,Dare Mighty Things,2017-01-01,...,NaN,NaN,NaN,YES,NaN,"Debut author Heather Kaczynski's new duology, ...",heather kaczynski,100,DARE MIGHTY THINGS NA UK,100
106,Angie Thomas,On the Come Up,2019-02-05,9780062498571,2019-02-05,9162855,8584,1134.668387,On the Come Up,NaT,...,NaN,NaN,NaN,NaN,NaN,"Angie Thomas's ON THE COME UP, to Moon (Nether...",angie thomas,100,ON THE COME UP,100
113,Alisha Rai,The Right Swipe,2019-08-06,9780062877864,2019-07-02,277994,278,997.362573,The Right Swipe,2019-01-01,...,NaN,NaN,NaN,NaN,NaN,"Alisha Rai's A PERFECT MATCH, in which two riv...",alisha rai,33,PERFECT MATCH,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1578,Ernest Cline,Ready player one,2018-03-15,9782749935881,2018-03-15,88736,64,1425.960784,Player one,2018-01-01,...,NaN,NaN,NaN,NaN,NaN,Author of Ready Player One and Armada Ernest C...,ernest cline,33,NA,77
1608,Rachel DeLoache Williams,My friend Anna,2019-08-19,9783641258191,2019-08-19,90683,86,970.983516,My Friend Anna,2019-01-01,...,NaN,NaN,NaN,NaN,NaN,"Rachel DeLoache Williams's MY FRIEND ANNA, to ...",rachel deloache williams,100,MY FRIEND ANNA,100
1664,Richard Wiseman,59 segundos,2015-03-26,9788490068519,2015-03-16,43883,52,874.266667,59 Seconds,2010-01-01,...,NaN,NaN,NaN,NaN,NaN,Media psychologist Richard Wiseman's SIXTY SEC...,richard wiseman,61,SIXTY SECONDS,76
1667,Richard Wiseman,59 segundos,2015-03-26,9788490068519,2018-08-28,11687,13,910.766667,59 Seconds,2010-01-01,...,NaN,NaN,NaN,NaN,NaN,Media psychologist Richard Wiseman's SIXTY SEC...,richard wiseman,61,SIXTY SECONDS,76


In [84]:
fm.columns
fm.shape

(1521, 28)

In [85]:
fm = fm[~fm['datepublished_y'].isna()]

In [86]:
fm.shape

(1449, 28)

In [87]:
fm

,isbn13,datepublished_x,totalrevenue,totalunits,price,author,title,datepublished_y,author_l,advance_id,...,Competition,Awards,Bestseller,Self Publishing,Debut,Series,All,Author_extracted,fuzz_ratio,title_extracted
0,9.780008e+12,2019-12-26,1480,1,1480.000000,Joanna Glen,The Other Half of Augusta Hope,2019-01-01,joanna glen,66000,...,YES,NaN,NaN,NaN,YES,NaN,Joanna Glen's debut THE FLIGHT OF AUGUSTA HOPE...,joanna glen,75,THE FLIGHT OF AUGUSTA HOPE UK
1,9.780008e+12,2020-01-01,0,0,1473.000000,Joanna Glen,The Other Half of Augusta Hope,2019-01-01,joanna glen,66000,...,YES,NaN,NaN,NaN,YES,NaN,Joanna Glen's debut THE FLIGHT OF AUGUSTA HOPE...,joanna glen,75,THE FLIGHT OF AUGUSTA HOPE UK
2,9.780008e+12,2020-03-05,25201,20,1309.250000,Joanna Glen,The Other Half of Augusta Hope,2019-01-01,joanna glen,66000,...,YES,NaN,NaN,NaN,YES,NaN,Joanna Glen's debut THE FLIGHT OF AUGUSTA HOPE...,joanna glen,75,THE FLIGHT OF AUGUSTA HOPE UK
3,9.780008e+12,2021-04-06,1899,1,1680.235294,Joe Hammond,A Short History of Falling,2021-01-01,joe hammond,57848,...,YES,NaN,NaN,NaN,NaN,NaN,Playwright Joe Hammond's A SHORT HISTORY OF FA...,joe hammond,100,SHORT HISTORY OF FALLING ALS
4,9.780062e+12,2017-10-03,1252305,1127,1048.526147,Marcie Colleen,"Love, Triangle",2017-01-01,marcie colleen,106679,...,YES,NaN,NaN,NaN,NaN,NaN,"Marcie Colleen's LOVE, TRIANGLE, about best fr...",marcie colleen,100,LOVE TRIANGLE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1516,9.781989e+12,2020-01-07,90107,101,874.060606,Jessica Cunsolo,She's With Me,2020-01-01,jessica cunsolo,52135,...,NaN,NaN,NaN,NaN,NaN,NaN,"Jessica Cunsolo's SHE'S WITH ME, reported as o...",jessica cunsolo,100,SHE WITH ME
1517,9.782214e+12,2018-09-26,35063,20,1880.473684,Amor Towles,Un gentleman à Moscou,2018-01-01,amor towles,90863,...,NaN,NaN,NaN,NaN,NaN,NaN,Rules of Civility author Amor Towles next two ...,amor towles,33,NA UK
1518,9.783641e+12,2018-03-05,539788,381,1425.031830,Bas Kast,Der Ernährungskompass,2018-01-01,bas kast,61933,...,YES,NaN,NaN,NaN,NaN,NaN,"Bas Kast's DER ERNAHRUNGSKOMPASS, to Business ...",bas kast,98,DER ERNAHRUNGSKOMPASS
1519,9.788420e+12,2018-01-18,769439,958,789.254743,Manuel Vilas,Ordesa,2018-01-01,manuel vilas,45720,...,NaN,NaN,NaN,NaN,NaN,NaN,"Manuel Vilas's ORDESA, to Writers Publishing H...",manuel vilas,100,ORDESA


In [80]:
fm = fm.groupby('isbn13', as_index=False).apply(lambda x : x.loc[x['datepublished_y'].idxmax()])


ValueError: attempt to get argmax of an empty sequence

In [88]:
print(fm.shape, fm['advance_id'].unique().shape, fm['isbn13'].unique().shape, fm['author_l'].unique().shape)


(1449, 28) (1052,) (1052,) (979,)


In [41]:
x = pd.merge(d, sales,on='isbn13')
x = x.groupby('isbn13',as_index=False).apply(lambda x: x.loc[x['datepublished_y'].idxmax()])

In [44]:
x

,author,title,datepublished_x,isbn13,author_l,datepublished_y,totalrevenue,totalunits,price
0,Steven Camden,It’s About Love,2015,9780007511259,steven camden,2015-06-04,60130,270,613.152174
1,Casey Watson,Runaway Girl: A beautiful girl. Trafficked for...,2016-10-20,9780008142599,casey watson,2016-10-20,1119609,1595,697.559242
2,Dolores Redondo,"Offering to the Storm (The Baztan Trilogy, Boo...",2017,9780008165550,dolores redondo,2017-08-24,160920,205,708.347305
3,Sam Carrington,Saving Sophie,2016-08-12,9780008191863,sam carrington,2018-08-12,299,1,299.000000
4,Katy Colins,Chasing the Sun,2017-07-27,9780008202200,katy colins,2017-07-27,36575,125,295.153846
...,...,...,...,...,...,...,...,...,...
3878,Anisur Rahman,Hazaron Khawaishen Aisi,2018,9789353023409,anisur rahman,2018-12-25,16516,27,606.246154
3879,Chris Skinner,Digital Bank: Strategies to launch or become a...,2014,9789814561808,chris skinner,2014-05-15,1437877,899,1609.589069
3880,Florencia Bonelli,"Dime, ¿quién es como Dios?",2019,9789877391305,florencia bonelli,2019-04-12,3297,3,1099.000000
3881,Mackenzi Lee,La guía del caballero para el vicio y la virtud,2015,9789877473315,mackenzi lee,2017-09-08,4194,6,710.111111


In [55]:
y = pd.merge(df, x, on=['author_l', 'title'])

In [56]:
y

,author_x,title,datepublished,isbn13_x,author_l,advance_id,Rights Category,Genre,Date,Author(s),...,Author_extracted,fuzz_ratio,title_extracted,author_y,datepublished_x,isbn13_y,datepublished_y,totalrevenue,totalunits,price
0,Malka Adler,The Brothers of Auschwitz,2019-01-01,9780008386115,malka adler,34526,International rights,Fiction,2020-10-12,NONE,...,malka adler,100,THE BROTHERS OF AUSCHWITZ UK,Malka Adler,2019,9780008386115,2019-11-07,1470355,11475,229.008850
1,Liana Liu,The Memory Key,2015-01-01,9780062306661,liana liu,124290,Children's,Young Adult Fiction,2013-03-26,Liana Liu,...,liana liu,100,MFA THE MEMORY KEY NA,Liana Liu,2015,9780062306661,2015-03-03,7182,18,403.333333
2,Suzanne Crowley,Finding Esme,2018-01-01,9780062352484,suzanne crowley,117324,Children's,Middle Grade Fiction,2013-11-22,Suzanne Crowley,...,suzanne crowley,100,FINDING ESME,Suzanne Crowley,2018,9780062352484,2018-08-14,18535,23,800.656250
3,Claire Needell,The First True Thing,2019-01-01,9780062360540,claire needell,62980,Children's,Young Adult Fiction,2018-07-06,Claire Needell,...,claire needell,100,THE WORD FOR YES THE FIRST TRUE THING,Claire Needell,2019,9780062360540,2019-04-23,11487,13,887.317003
4,Roxane Gay,Not That Bad,2018-01-01,9780062413505,roxane gay,39661,International rights,Non-fiction,2020-05-15,NONE,...,roxane gay,100,NOT THAT BAD,Roxane Gay,2018,9780062413505,2023-11-06,0,0,853.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,Brian Banks,What Set Me Free (The Story That Inspired the ...,2019-01-01,9781982121327,brian banks,60101,Non-fiction,General/ Other,2018-09-29,Brian Banks,...,brian banks,43,NFL THE BRIAN BANKS STORY NFL,Brian Banks,2019,9781982121327,2019-07-02,526512,409,1277.327645
561,Leslie Nagel,The Codebook Murders,2019-01-01,9781984800268,leslie nagel,95236,Digital Fiction,Mystery/ Crime,2015-12-02,Leslie Nagel,...,leslie nagel,78,THE BOOK CLUB MURDERS UK,Leslie Nagel,2019,9781984800268,2019-05-21,1103078,3355,491.674944
562,Joel Dane,Kill Orbit,2020-01-01,9781984802576,joel dane,65841,Fiction,Sci-Fi/ Fantasy,2018-04-12,Joel Dane,...,joel dane,49,CRY PILOT BURN CYCLE KILL ORBIT SF,Joel Dane,2020,9781984802576,2020-07-07,69537,81,889.858586
563,Amor Towles,Un gentleman à Moscou,2018-01-01,9782213704296,amor towles,90863,Fiction,General/ Other,2016-04-13,Amor Towles,...,amor towles,33,NA UK,Amor Towles,2018,9782213704296,2018-09-26,35063,20,1880.473684


In [54]:
y.columns

Index(['author_x', 'title_x', 'datepublished', 'isbn13_x', 'author_l',
       'advance_id', 'Rights Category', 'Genre', 'Date', 'Author(s)', 'Title',
       'Publishers', 'Big Publishing House Affilation', 'Advance',
       'Competition', 'Awards', 'Bestseller', 'Self Publishing', 'Debut',
       'Series', 'All', 'Author_extracted', 'fuzz_ratio', 'title_extracted',
       'author_y', 'title_y', 'datepublished_x', 'isbn13_y', 'datepublished_y',
       'totalrevenue', 'totalunits', 'price'],
      dtype='object')

In [53]:
y['fuzz_ratio_1'] = book_adv.apply(lambda x : levenshtein_ratio(str(x['title_']).lower(), str(x['title']).lower()), axis=1)

,author_x,title_x,datepublished,isbn13_x,author_l,advance_id,Rights Category,Genre,Date,Author(s),...,fuzz_ratio,title_extracted,author_y,title_y,datepublished_x,isbn13_y,datepublished_y,totalrevenue,totalunits,price
0,Steven Camden,Tape,2014-01-01,9780007511211,steven camden,118703,International rights,UK Fiction,2013-10-08,Steven Camden,...,100,TAPE UK,Steven Camden,It’s About Love,2015,9780007511259,2015-06-04,60130,270,613.152174
1,Alexander McCall Smith,Emma,2015-01-01,9780007553884,alexander mccall smith,118748,Fiction,General/ Other,2013-10-07,Alexander McCall Smith,...,100,EMMA,Alexander McCall Smith,The Sands of Shark Island,2017-07-11,9780399554032,2017-07-11,82523,78,1066.443719
2,Alexander McCall Smith,Emma,2015-01-01,9780007553884,alexander mccall smith,118748,Fiction,General/ Other,2013-10-07,Alexander McCall Smith,...,100,EMMA,Alexander McCall Smith,Dream Angus,2012,9780802194220,2019-04-24,119274,426,457.742222
3,Alexander McCall Smith,Corduroy Mansions,NaT,9780307398345,alexander mccall smith,150523,Fiction,General/ Other,2010-05-25,NONE,...,100,CORDUROY MANSIONS SCOTLAND STREET,Alexander McCall Smith,The Sands of Shark Island,2017-07-11,9780399554032,2017-07-11,82523,78,1066.443719
4,Alexander McCall Smith,Corduroy Mansions,NaT,9780307398345,alexander mccall smith,150523,Fiction,General/ Other,2010-05-25,NONE,...,100,CORDUROY MANSIONS SCOTLAND STREET,Alexander McCall Smith,Dream Angus,2012,9780802194220,2019-04-24,119274,426,457.742222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4185,Troy Cummings,Troy Cummings's the Binder of Doom Collection,2022-01-01,9789813383531,troy cummings,20827,Children's,Picture Book Fiction,2021-09-28,Troy Cummings,...,35,NYT CAN BE YOUR DOG OTTO THE ORNAMENT,Troy Cummings,Boa Constructor: A Branches Book (The Binder o...,2019,9781338314717,2019-09-03,89545,363,249.735043
4186,Aisha Saeed,Le Courage d'Amal,2021-01-01,9791023514698,aisha saeed,53030,Children's,Young Adult Fiction,2019-04-17,"['Becky Albertalli', 'Aisha Saeed']",...,44,NYT YES NO MAYBE SO,Aisha Saeed,Aladdin: Far From Agrabah,2019,9781368047425,2019-09-03,1099,1,1099.000000
4187,Jaime Lorente,Basho and the Pattern 5-7-5,2023-01-01,9798376140987,jaime lorente,26465,International rights,Fiction,2021-05-04,NONE,...,33,ABOUT YOUR MOUTH,Jaime Lorente,A propósito de tu boca,2019,9788467055542,2019-03-19,26071,29,856.190476
4188,Olivia Miles,A Promise to Keep,2023-01-01,9798986262451,olivia miles,11138,Audio rights,NONE,2022-05-20,Olivia Miles,...,33,PLACE FOR US SECOND CHANCE SUMMER BECAUSE OF YOU,Olivia Miles,The Winter Wedding Plan,2017,9781455567249,2017-09-26,2889301,4421,639.829344


## Ethnicity Prediction of Author

In [74]:
from ethnicolr import pred_fl_reg_name

In [75]:
def preprocess_names(df, Column_Name):
    #Remove space if the first letter of the word is a space
    df['temp_Column_Name'] = df[Column_Name]
    df['temp_Column_Name'] = df['temp_Column_Name'].str.lstrip()
    
    #Remove single letter in the Names
    #df[Column_Name] = df[Column_Name].str.replace(r'\b[A-Za-z]\b', '')
    df['temp_Column_Name'] = df['temp_Column_Name'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 1]))
    #Remove names in brackets
    #df[Column_Name] = df[Column_Name].str.replace(r'\s?\([^()]*\)', '')
    df['temp_Column_Name'] = df['temp_Column_Name'].apply(lambda x: re.sub("[\(\[].*?[\)\]]", "", x))
    
    rwords = ['Doctor','Professor','Captain','Jr','Sr','III','II', 'IV']
    for word in rwords:
        df.loc[:, 'temp_Column_Name'] = df['temp_Column_Name'].str.replace(word,"")
        
    df['temp_Column_Name'] = df['temp_Column_Name'].str.lstrip()
    
    df['First Name'] = df['temp_Column_Name'].str.split(" ", expand=True)[0]
    df['Last Name'] = df['temp_Column_Name'].str.split().str[-1]
    df.drop(columns=['temp_Column_Name'], inplace=True)
    return df

In [76]:
fm

,isbn13,datepublished_x,totalrevenue,totalunits,price,author,title,datepublished_y,author_l,advance_id,...,Competition,Awards,Bestseller,Self Publishing,Debut,Series,All,Author_extracted,fuzz_ratio,title_extracted
0,9.780008e+12,2020-03-05,25201,20,1309.250000,Joanna Glen,The Other Half of Augusta Hope,2019-01-01,joanna glen,66000,...,YES,NaN,NaN,NaN,YES,NaN,Joanna Glen's debut THE FLIGHT OF AUGUSTA HOPE...,joanna glen,75,THE FLIGHT OF AUGUSTA HOPE UK
1,9.780008e+12,2021-04-06,1899,1,1680.235294,Joe Hammond,A Short History of Falling,2021-01-01,joe hammond,57848,...,YES,NaN,NaN,NaN,NaN,NaN,Playwright Joe Hammond's A SHORT HISTORY OF FA...,joe hammond,100,SHORT HISTORY OF FALLING ALS
2,9.780062e+12,2017-10-03,1252305,1127,1048.526147,Marcie Colleen,"Love, Triangle",2017-01-01,marcie colleen,106679,...,YES,NaN,NaN,NaN,NaN,NaN,"Marcie Colleen's LOVE, TRIANGLE, about best fr...",marcie colleen,100,LOVE TRIANGLE
3,9.780062e+12,2019-06-25,10188,6,1698.000000,Alissa Quart,Squeezed,2019-01-01,alissa quart,60539,...,YES,NaN,NaN,NaN,NaN,NaN,"Alissa Quart's SQUEEZED, to Gusa (Taiwan, Prov...",alissa quart,100,SQUEEZED
4,9.780063e+12,2018-07-24,1492131,1364,1070.016355,Erin Bowman,Contagion,2018-01-01,erin bowman,82642,...,NaN,NaN,NaN,NaN,NaN,YES,VENGEANCE ROAD author Erin Bowman's sci-fi CON...,erin bowman,100,VENGEANCE ROAD CONTAGION NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590,9.798501e+12,2021-05-08,1399,1,1399.000000,Aj Vanderhorst,The Mostly Invisible Boy (Casey Grimes),2021-01-01,aj vanderhorst,46214,...,NaN,NaN,NaN,NaN,YES,NaN,AJ Vanderhorst's debut THE MOSTLY INVISIBLE BO...,aj vanderhorst,79,AJ THE MOSTLY INVISIBLE BOY US
591,9.798617e+12,2020-02-25,19481,19,1023.444444,Christopher Ruocchio,The Lesser Devil,2020-01-01,christopher ruocchio,93513,...,NaN,NaN,NaN,NaN,NaN,NaN,"Christopher Ruocchio's THE MURDERED SUM, Book ...",christopher ruocchio,50,THE MURDERED SUM DAW NA UK
592,9.798642e+12,2020-04-30,487017,351,1390.812834,Shannon Mayer,Midlife Bounty Hunter,NaT,shannon mayer,35513,...,NaN,NaN,NaN,NaN,NaN,NaN,"Shannon Mayer's MIDLIFE BOUNTY HUNTER, to Eksm...",shannon mayer,100,MIDLIFE BOUNTY HUNTER
593,9.798682e+12,2020-09-03,1926121,1223,1593.222222,Carrie Aarons,Love at First Fight,NaT,carrie aarons,24416,...,NaN,NaN,NaN,NaN,NaN,NaN,"Carrie Aarons's LOVE AT FIRST FIGHT, to Tantor...",carrie aarons,100,LOVE AT FIRST FIGHT


In [77]:
dg = preprocess_names(fm, 'author_l')


In [78]:
dg1 = dg.copy()

In [79]:
eth = pred_fl_reg_name(dg,'Last Name', 'First Name')

In [80]:
x = dg1.merge(eth, on=['First Name', 'Last Name'], how='outer')

In [81]:
x.columns

Index(['isbn13_x', 'datepublished_x_x', 'totalrevenue_x', 'totalunits_x',
       'price_x', 'author_x', 'title_x', 'datepublished_y_x', 'author_l_x',
       'advance_id_x', 'Rights Category_x', 'Genre_x', 'Date_x', 'Author(s)_x',
       'Title_x', 'Publishers_x', 'Big Publishing House Affilation_x',
       'Advance_x', 'Competition_x', 'Awards_x', 'Bestseller_x',
       'Self Publishing_x', 'Debut_x', 'Series_x', 'All_x',
       'Author_extracted_x', 'fuzz_ratio_x', 'title_extracted_x', 'First Name',
       'Last Name', 'isbn13_y', 'datepublished_x_y', 'totalrevenue_y',
       'totalunits_y', 'price_y', 'author_y', 'title_y', 'datepublished_y_y',
       'author_l_y', 'advance_id_y', 'Rights Category_y', 'Genre_y', 'Date_y',
       'Author(s)_y', 'Title_y', 'Publishers_y',
       'Big Publishing House Affilation_y', 'Advance_y', 'Competition_y',
       'Awards_y', 'Bestseller_y', 'Self Publishing_y', 'Debut_y', 'Series_y',
       'All_y', 'Author_extracted_y', 'fuzz_ratio_y', 'title_ext

In [82]:
x = x[[column for column in x.columns if not column.endswith('_y')]]


In [83]:
x.columns  = x.columns.str.replace('_x','')

In [84]:
x.columns

Index(['isbn13', 'datepublished', 'totalrevenue', 'totalunits', 'price',
       'author', 'title', 'datepublished_y', 'author_l', 'advance_id',
       'Rights Category', 'Genre', 'Date', 'Author(s)', 'Title', 'Publishers',
       'Big Publishing House Affilation', 'Advance', 'Competition', 'Awards',
       'Bestseller', 'Self Publishing', 'Debut', 'Series', 'All',
       'Author_extracted', 'fuzz_ratio', 'title_extracted', 'First Name',
       'Last Name', 'asian', 'hispanic', 'nh_black', 'nh_white', 'race'],
      dtype='object')

In [85]:
x = x.rename(columns={
    
})

In [60]:
x.to_csv('SGAADV_ebook.csv')

In [86]:
x.columns

Index(['isbn13', 'datepublished', 'totalrevenue', 'totalunits', 'price',
       'author', 'title', 'datepublished_y', 'author_l', 'advance_id',
       'Rights Category', 'Genre', 'Date', 'Author(s)', 'Title', 'Publishers',
       'Big Publishing House Affilation', 'Advance', 'Competition', 'Awards',
       'Bestseller', 'Self Publishing', 'Debut', 'Series', 'All',
       'Author_extracted', 'fuzz_ratio', 'title_extracted', 'First Name',
       'Last Name', 'asian', 'hispanic', 'nh_black', 'nh_white', 'race'],
      dtype='object')

In [87]:
x['datepublished_month'] = x['datepublished'].dt.strftime('%Y-%m')

In [89]:
x.groupby('datepublished_month')[['totalrevenue', 'Advance','isbn13']].sum().to_csv('printbook_datewise.csv')